# Market Data Analysis - Getting Started

This notebook demonstrates how to use the Market Data Fetcher to download and analyze stock market data.

## Setup and Imports

In [ ]:
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

# Import our modules
from src.data_sources.nse_fetcher import NSEFetcher
from src.data_sources.web_scraper import WebScraper
from src.utils.data_manager import DataManager
from src.utils.config_manager import ConfigManager

# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Configure plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

print("Setup complete!")

## Initialize Data Sources

In [ ]:
# Initialize data sources
nse_fetcher = NSEFetcher()
web_scraper = WebScraper()
data_manager = DataManager()
config = ConfigManager()

print("Data sources initialized!")

## Fetch NSE Index Data

In [ ]:
# Get NIFTY index data
nifty_data = nse_fetcher.get_index_data('NIFTY')

if nifty_data:
    print(f"NIFTY 50 Current Value: {nifty_data.get('lastPrice', 'N/A')}")
    print(f"Change: {nifty_data.get('change', 'N/A')} ({nifty_data.get('pChange', 'N/A')}%)")
    print(f"High: {nifty_data.get('dayHigh', 'N/A')}")
    print(f"Low: {nifty_data.get('dayLow', 'N/A')}")
else:
    print("Failed to fetch NIFTY data")

## Get Top Gainers and Losers

In [ ]:
# Get top gainers
top_gainers = nse_fetcher.get_top_gainers(5)

if top_gainers:
    print("Top 5 Gainers:")
    gainers_df = pd.DataFrame(top_gainers)
    display(gainers_df[['symbol', 'lastPrice', 'change', 'pChange']].head())
else:
    print("Failed to fetch top gainers")

# Get top losers
top_losers = nse_fetcher.get_top_losers(5)

if top_losers:
    print("\nTop 5 Losers:")
    losers_df = pd.DataFrame(top_losers)
    display(losers_df[['symbol', 'lastPrice', 'change', 'pChange']].head())
else:
    print("Failed to fetch top losers")

## Fetch Historical Data using Yahoo Finance

In [ ]:
# Get historical data for RELIANCE
symbol = 'RELIANCE.NS'
historical_data = web_scraper.get_yahoo_finance_data(symbol, period='3mo')

if historical_data is not None and not historical_data.empty:
    print(f"Downloaded {len(historical_data)} days of data for {symbol}")
    print("\nFirst 5 rows:")
    display(historical_data.head())
    
    print("\nLast 5 rows:")
    display(historical_data.tail())
else:
    print(f"Failed to fetch data for {symbol}")

## Visualize Stock Price Data

In [ ]:
if historical_data is not None and not historical_data.empty:
    # Create subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    # Plot closing price
    ax1.plot(historical_data.index, historical_data['Close'], linewidth=2, label='Close Price')
    ax1.set_title(f'{symbol} - Stock Price (Last 3 Months)')
    ax1.set_ylabel('Price (INR)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot volume
    ax2.bar(historical_data.index, historical_data['Volume'], alpha=0.6, color='orange')
    ax2.set_title(f'{symbol} - Trading Volume')
    ax2.set_ylabel('Volume')
    ax2.set_xlabel('Date')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate basic statistics
    print(f"\nBasic Statistics for {symbol}:")
    print(f"Current Price: ₹{historical_data['Close'].iloc[-1]:.2f}")
    print(f"3-Month High: ₹{historical_data['High'].max():.2f}")
    print(f"3-Month Low: ₹{historical_data['Low'].min():.2f}")
    print(f"Average Volume: {historical_data['Volume'].mean():,.0f}")
    
    # Calculate returns
    returns = historical_data['Close'].pct_change().dropna()
    print(f"\nReturn Statistics:")
    print(f"Average Daily Return: {returns.mean()*100:.2f}%")
    print(f"Volatility (Std Dev): {returns.std()*100:.2f}%")
    print(f"3-Month Return: {((historical_data['Close'].iloc[-1] / historical_data['Close'].iloc[0]) - 1)*100:.2f}%")

## Compare Multiple Stocks

In [ ]:
# Compare multiple stocks
symbols = ['RELIANCE.NS', 'TCS.NS', 'INFY.NS', 'HDFC.NS']
comparison_data = web_scraper.get_multiple_stocks_data(symbols, period='1mo')

if comparison_data:
    # Create comparison chart
    plt.figure(figsize=(12, 8))
    
    for symbol, data in comparison_data.items():
        if not data.empty:
            # Normalize to percentage change
            normalized = (data['Close'] / data['Close'].iloc[0] - 1) * 100
            plt.plot(normalized.index, normalized, linewidth=2, label=symbol.replace('.NS', ''))
    
    plt.title('Stock Performance Comparison (1 Month) - Normalized')
    plt.xlabel('Date')
    plt.ylabel('Percentage Change (%)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.axhline(y=0, color='black', linestyle='-', alpha=0.5)
    plt.tight_layout()
    plt.show()
    
    # Performance summary
    print("\n1-Month Performance Summary:")
    for symbol, data in comparison_data.items():
        if not data.empty:
            performance = ((data['Close'].iloc[-1] / data['Close'].iloc[0]) - 1) * 100
            print(f"{symbol.replace('.NS', '')}: {performance:.2f}%")

## Get Market News

In [ ]:
# Get latest market news
news = web_scraper.get_market_news()

if news:
    print(f"Found {len(news)} news articles:")
    print("\nLatest Market News:")
    
    for i, article in enumerate(news[:5]):
        print(f"\n{i+1}. {article.get('title', 'No title')}")
        print(f"   Source: {article.get('source', 'Unknown')}")
        if article.get('summary'):
            print(f"   Summary: {article['summary'][:100]}...")
else:
    print("No news articles found")

## Currency Rates

In [ ]:
# Get currency rates
rates = web_scraper.get_currency_rates()

if rates:
    print("Current Currency Rates:")
    for pair, rate in rates.items():
        print(f"{pair}: {rate}")
else:
    print("Failed to fetch currency rates")

## Store Data in Database

In [ ]:
# Store historical data in database
if historical_data is not None and not historical_data.empty:
    # Add symbol column
    historical_data_copy = historical_data.copy()
    historical_data_copy['symbol'] = symbol.replace('.NS', '')
    historical_data_copy['date'] = historical_data_copy.index.date
    
    # Store in database
    data_manager.store_stock_data(historical_data_copy, source='yahoo_finance')
    print(f"Stored {len(historical_data_copy)} records for {symbol} in database")

# Store index data
if nifty_data:
    data_manager.store_index_data(
        index_name='NIFTY',
        value=float(nifty_data.get('lastPrice', 0)),
        change_points=float(nifty_data.get('change', 0)),
        change_percent=float(nifty_data.get('pChange', 0)),
        source='nse'
    )
    print("Stored NIFTY index data in database")

# Get data summary
summary = data_manager.get_data_summary()
print("\nDatabase Summary:")
for key, value in summary.items():
    print(f"{key}: {value}")

## Export Data

In [ ]:
# Add a stock to watchlist
data_manager.add_to_watchlist('RELIANCE', name='Reliance Industries Limited', exchange='NSE', sector='Energy')
data_manager.add_to_watchlist('TCS', name='Tata Consultancy Services', exchange='NSE', sector='IT')
data_manager.add_to_watchlist('INFY', name='Infosys Limited', exchange='NSE', sector='IT')

# Get watchlist
watchlist = data_manager.get_watchlist()
print("Current Watchlist:")
for item in watchlist:
    print(f"- {item['symbol']}: {item['name']}")

# Export data to CSV
export_path = data_manager.export_to_csv('RELIANCE')
if export_path:
    print(f"\nData exported to: {export_path}")
else:
    print("\nNo data available for export")

## Summary

This notebook demonstrated:
1. Setting up and initializing the market data fetcher
2. Fetching real-time data from NSE
3. Getting historical data using Yahoo Finance
4. Visualizing stock price movements and volume
5. Comparing multiple stocks
6. Retrieving market news
7. Getting currency exchange rates
8. Storing data in the database
9. Managing watchlists and exporting data

You can extend this notebook to:
- Add technical indicators
- Perform more advanced analysis
- Create automated reporting
- Integrate with Zerodha API for trading data
- Build custom alerts and notifications